# tensorboard可视化界面

In [14]:
import tensorflow as tf
import datetime
import os

In [15]:
%load_ext tensorboard
%matplotlib inline

In [6]:
(train_img, train_label), (test_img, test_label) = tf.keras.datasets.fashion_mnist.load_data()

In [7]:
train_img = train_img  / 255

test_img = test_img / 255

ds_train_img = tf.data.Dataset.from_tensor_slices(train_img)
ds_train_lab = tf.data.Dataset.from_tensor_slices(train_label)
ds_test_img = tf.data.Dataset.from_tensor_slices(test_img)
ds_test_lab = tf.data.Dataset.from_tensor_slices(test_label)

ds_train = tf.data.Dataset.zip((ds_train_img, ds_train_lab))
ds_test = tf.data.Dataset.zip((ds_test_img, ds_test_lab))

train_dataset = ds_train.shuffle(10000).repeat().batch(64)
test_dataset = ds_test.batch(64)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [9]:
model.compile(
    optimizer="adam",
    metrics=["accuracy"],
    loss="sparse_categorical_crossentropy"
)

In [10]:
validation_steps = test_img.shape[0]//64
steps_per_epoch = train_img.shape[0]//64

In [12]:
log_dir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [13]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

### 自定义变量的tensorboard的可视化

In [31]:
# 创建一个文件编辑器
file_writer = tf.summary.create_file_writer(log_dir + '/lr')
# 设置为默认的编辑器
file_writer.set_as_default()

In [32]:
def lr_rate(epoch):
    learning_rate = 0.02
    if epoch > 5:
        learning_rate = 0.01
    if epoch > 10:
        learning_rate = 0.005
    if epoch > 15:
        learning_rate = 0.0005
    tf.summary.scalar("learning_rate", learning_rate, step=epoch)
    return learning_rate

In [33]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_rate)

这里我们需要把我们记录的文件写入到磁盘中

In [34]:
model.fit(
    train_dataset, 
    epochs=20, 
    steps_per_epoch=steps_per_epoch, 
    validation_data=test_dataset, 
    validation_steps=validation_steps,
    callbacks=[tensorboard_callback, lr_callback]
)

Train for 937 steps, validate for 156 steps
Epoch 1/20
937/937 [==============================] - 2s 2ms/step - loss: 2.3048 - accuracy: 0.0986 - val_loss: 2.3041 - val_accuracy: 0.1001
Epoch 2/20
937/937 [==============================] - 2s 2ms/step - loss: 2.3043 - accuracy: 0.1012 - val_loss: 2.3060 - val_accuracy: 0.0999
Epoch 3/20
937/937 [==============================] - 2s 2ms/step - loss: 2.3046 - accuracy: 0.0970 - val_loss: 2.3043 - val_accuracy: 0.1001
Epoch 4/20
937/937 [==============================] - 2s 2ms/step - loss: 2.3048 - accuracy: 0.0985 - val_loss: 2.3055 - val_accuracy: 0.1001
Epoch 5/20
937/937 [==============================] - 2s 2ms/step - loss: 2.3046 - accuracy: 0.0987 - val_loss: 2.3057 - val_accuracy: 0.1001
Epoch 6/20
937/937 [==============================] - 2s 2ms/step - loss: 2.3045 - accuracy: 0.1006 - val_loss: 2.3045 - val_accuracy: 0.1001
Epoch 7/20
937/937 [==============================] - 2s 2ms/step - loss: 2.3035 - accuracy: 0.1007 - va

In [35]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 83636), started 1:35:02 ago. (Use '!kill 83636' to kill it.)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
